In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [27]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [13]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [14]:
#CLEAN TRAINING DATA
tx_clean = remove_undefined_values (tX)

In [15]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
y, tx = build_model_data(tx,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)

In [16]:
num_samples, num_features
tx.shape

(250000, 31)

In [39]:
from compute_gradient import *
from cost import *
from implementations import *

In [40]:
#Initial weight
initial_w = np.zeros((num_features,1))

#Maximum iterations through the whole set
max_iter = 100

#Step-size
gamma = 0.001

#Regularization factor
lambda_ = 0.01

In [ ]:
weights, loss = reg_logistic_regression(y, tx, lambda_, initial_w, max_iter, gamma)

Gradient Descent(0/99): loss=0.2340171397964098
Gradient Descent(1/99): loss=1.7994215461480565
Gradient Descent(2/99): loss=2.0474601297826074
Gradient Descent(3/99): loss=0.19167329011877088
Gradient Descent(4/99): loss=0.4494830515497463
Gradient Descent(5/99): loss=0.4628053161698752


In [22]:
w

array([[-0.91674615],
       [ 0.05125081],
       [-0.6523374 ],
       [-1.15162356],
       [ 0.40409732],
       [-0.06942197],
       [ 0.4358453 ],
       [ 0.10428752],
       [ 1.08605389],
       [-0.1244216 ],
       [ 0.10017966],
       [-0.80158974],
       [ 0.25493044],
       [ 0.2322356 ],
       [ 0.45257223],
       [-0.01839999],
       [ 0.04081387],
       [ 1.05019472],
       [ 0.02487006],
       [ 0.01687931],
       [ 0.13886227],
       [-0.01603594],
       [-0.14709146],
       [ 0.14744692],
       [-0.11353822],
       [ 0.03083295],
       [-0.03162034],
       [-0.12346362],
       [ 0.02806428],
       [-0.02153869],
       [-0.22159283]])

In [21]:
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [22]:
tx_test_clean = remove_undefined_values (tX_test)
tx_test, _, _ = standardize(tx_test_clean,0)


In [23]:
ytest, tx_test = build_model_data(tx_test,ytest)

In [24]:
OUTPUT_PATH = 'data/submission_reg_logistic.csv'
y_pred = predict_labels(w, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)